In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Date, DECIMAL, ForeignKey
from sqlalchemy.orm import Session

In [2]:
# Import data into data frames and rename columns in preparation for storage.
measures = pd.read_csv('hawaii_measurements.csv', names=('station_name', 'date', 'precipitation', 'temperature'), header=0, parse_dates=[1])
stations = pd.read_csv('hawaii_stations.csv', names=('station_name', 'location_name', 'latitude', 'longitude', 'elevation'), header=0)

In [3]:
measures.head()

,station_name,date,precipitation,temperature
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [4]:
measures.dtypes

station_name             object
date             datetime64[ns]
precipitation           float64
temperature               int64
dtype: object

In [5]:
# There are 1,447 incomplete cases in "measures" due to NaN values in measures['prcp'].

measures.isnull().sum()

station_name        0
date                0
precipitation    1447
temperature         0
dtype: int64

In [6]:
# There are no NaN values in "stations"

stations

,station_name,location_name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [7]:
# If complete cases are needed, Pandas' .dropna() can be used to elimate all rows with NaNs.
# Here a "clean" copy of the data is saved to a new CSV.

measures_clean = measures.dropna(axis=0)
measures_clean.to_csv("hawaii_measurements_clean.csv")

In [8]:
# Create engine and connection to SQLite.

engine = create_engine("sqlite:///hawaii_weather.sqlite")
conn = engine.connect()


In [9]:
base = declarative_base()

In [10]:
class station(base):
    __tablename__ = 'station'
    
    station_name = Column(String(255), primary_key=True)
    location_name = Column(String(255))
    latitude = Column(Numeric)
    longitude = Column(Numeric)
    elevation = Column(Numeric)
    
    def __repr__(self):
        return '<station(%r, %r)>' % (self.station_name, self.location_name)

class measurement(base):
    __tablename__ = 'measurement'
    
    id = Column(Integer, primary_key=True)
    station_name = Column(String(255), ForeignKey('station.station_name'))
    date = Column(Date)
#     precipitation = Column(Numeric, nullable=True)
    precipitation = Column(String(255), nullable=True)
    temperature = Column(Numeric, nullable=True)
    
    def __repr__(self):
        return "<measure(%r, %r)>" % (self.station_name, self.date)

In [11]:
station.__table__

Table('station', MetaData(bind=None), Column('station_name', String(length=255), table=<station>, primary_key=True, nullable=False), Column('location_name', String(length=255), table=<station>), Column('latitude', Numeric(), table=<station>), Column('longitude', Numeric(), table=<station>), Column('elevation', Numeric(), table=<station>), schema=None)

In [12]:
measurement(station_name = 'jake', date = '2008-01-01', precipitation = 12, temperature = 55)

<measure('jake', '2008-01-01')>

In [13]:
measurement.__table__

Table('measurement', MetaData(bind=None), Column('id', Integer(), table=<measurement>, primary_key=True, nullable=False), Column('station_name', String(length=255), ForeignKey('station.station_name'), table=<measurement>), Column('date', Date(), table=<measurement>), Column('precipitation', String(length=255), table=<measurement>), Column('temperature', Numeric(), table=<measurement>), schema=None)

In [14]:
base.metadata.create_all(engine)

In [15]:
stations_dict = stations.to_dict(orient='records')
stations_dict[0]

{'elevation': 3.0,
 'latitude': 21.2716,
 'location_name': 'WAIKIKI 717.2, HI US',
 'longitude': -157.8168,
 'station_name': 'USC00519397'}

In [16]:
measures_dict = measures.to_dict(orient='records')
measures_dict

[{'date': Timestamp('2010-01-01 00:00:00'),
  'precipitation': 0.08,
  'station_name': 'USC00519397',
  'temperature': 65},
 {'date': Timestamp('2010-01-02 00:00:00'),
  'precipitation': 0.0,
  'station_name': 'USC00519397',
  'temperature': 63},
 {'date': Timestamp('2010-01-03 00:00:00'),
  'precipitation': 0.0,
  'station_name': 'USC00519397',
  'temperature': 74},
 {'date': Timestamp('2010-01-04 00:00:00'),
  'precipitation': 0.0,
  'station_name': 'USC00519397',
  'temperature': 76},
 {'date': Timestamp('2010-01-06 00:00:00'),
  'precipitation': nan,
  'station_name': 'USC00519397',
  'temperature': 73},
 {'date': Timestamp('2010-01-07 00:00:00'),
  'precipitation': 0.06,
  'station_name': 'USC00519397',
  'temperature': 70},
 {'date': Timestamp('2010-01-08 00:00:00'),
  'precipitation': 0.0,
  'station_name': 'USC00519397',
  'temperature': 64},
 {'date': Timestamp('2010-01-09 00:00:00'),
  'precipitation': 0.0,
  'station_name': 'USC00519397',
  'temperature': 68},
 {'date': Time

In [17]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [18]:
# station_table = sqlalchemy.Table('station', metadata, autoload=True)
# measurement_table = sqlalchemy.Table('measurement', metadata, autoload=True)

In [19]:
# conn.execute(station_table.insert(), stations_dict)

In [20]:
# conn.execute("select * from station limit 5").fetchall()

In [21]:
# measures_objects = conn.execute(measurement_table.insert(), measures_dict)

In [22]:
# conn.execute("select * from measurement limit 5").fetchall()

In [23]:
station_obj = [station(station_name=stations_dict[i]['station_name'],
                       location_name=stations_dict[i]['location_name'],
                       latitude=stations_dict[i]['latitude'],
                       longitude=stations_dict[i]['longitude'],
                       elevation=stations_dict[i]['elevation']
                      ) for i in range(0,len(stations_dict))
              ]
station_obj

[<station('USC00519397', 'WAIKIKI 717.2, HI US')>,
 <station('USC00513117', 'KANEOHE 838.1, HI US')>,
 <station('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US')>,
 <station('USC00517948', 'PEARL CITY, HI US')>,
 <station('USC00518838', 'UPPER WAHIAWA 874.3, HI US')>,
 <station('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US')>,
 <station('USC00519281', 'WAIHEE 837.5, HI US')>,
 <station('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US')>,
 <station('USC00516128', 'MANOA LYON ARBO 785.2, HI US')>]

In [24]:
measurement_object = [measurement(station_name = measures_dict[i]['station_name'],
                                  date = measures_dict[i]['date'],
                                  precipitation = measures_dict[i]['precipitation'],
                                  temperature = measures_dict[i]['temperature']
                                 ) for i in range(0, len(measures_dict))
                     ]
measurement_object

[<measure('USC00519397', Timestamp('2010-01-01 00:00:00'))>,
 <measure('USC00519397', Timestamp('2010-01-02 00:00:00'))>,
 <measure('USC00519397', Timestamp('2010-01-03 00:00:00'))>,
 <measure('USC00519397', Timestamp('2010-01-04 00:00:00'))>,
 <measure('USC00519397', Timestamp('2010-01-06 00:00:00'))>,
 <measure('USC00519397', Timestamp('2010-01-07 00:00:00'))>,
 <measure('USC00519397', Timestamp('2010-01-08 00:00:00'))>,
 <measure('USC00519397', Timestamp('2010-01-09 00:00:00'))>,
 <measure('USC00519397', Timestamp('2010-01-10 00:00:00'))>,
 <measure('USC00519397', Timestamp('2010-01-11 00:00:00'))>,
 <measure('USC00519397', Timestamp('2010-01-12 00:00:00'))>,
 <measure('USC00519397', Timestamp('2010-01-14 00:00:00'))>,
 <measure('USC00519397', Timestamp('2010-01-15 00:00:00'))>,
 <measure('USC00519397', Timestamp('2010-01-16 00:00:00'))>,
 <measure('USC00519397', Timestamp('2010-01-17 00:00:00'))>,
 <measure('USC00519397', Timestamp('2010-01-18 00:00:00'))>,
 <measure('USC00519397',

In [25]:
session = Session(bind=engine)

In [26]:
session.rollback()
session.bulk_save_objects(station_obj)
session.bulk_save_objects(measurement_object)

session.commit()

In [27]:
conn.close()
engine.dispose()